# fetch data

In [117]:
import requests
import csv
from datetime import datetime, timedelta

api_url = "https://neptune.kyogojo.com/api/statistics/get-multiple?stations=BSLG-003&days=1740441600000,1740528000000"

response = requests.get(api_url)
data = response.json() 

pressure_data = data["payload"]["data"][0]["pressure"]

csv_filename = r"C:\Users\Sarah\Desktop\trend_analysis\data\water_pressure.csv"

# save to CSV
with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["value", "time"])

    for entry in pressure_data:
        value = entry["value"]
        timestamp = entry["time"]
        
        # convert UTC time to Philippine Standard Time (UTC+8)
        utc_time = datetime.utcfromtimestamp(timestamp / 1000)  #milliseconds to seconds
        pst_time = utc_time + timedelta(hours=8)  

        formatted_time = pst_time.strftime("%Y-%m-%d %H:%M:%S")    
        writer.writerow([value, formatted_time])

print(f"Data saved to {csv_filename}")


Data saved to C:\Users\Sarah\Desktop\trend_analysis\data\water_pressure.csv


C:\Users\Sarah\AppData\Local\Temp\ipykernel_14284\1108656560.py:24: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  utc_time = datetime.utcfromtimestamp(timestamp / 1000)  #milliseconds to seconds


# MERGE DATA

In [144]:
import pandas as pd

# Load the data BSLG-002
file_path = r"C:\Users\Sarah\Desktop\trend_analysis\data\BSLG-002-JAN-MAR.csv"
df = pd.read_csv(file_path)

# Define the ranges
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
labels = ['0-10', '11-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89']

# Create a new column for the ranges
df['Value Range'] = pd.cut(df['value'], bins=bins, labels=labels, right=False)

# Count occurrences in each range
range_counts = df['Value Range'].value_counts()

print(range_counts)


Value Range
50-59    39371
40-49    30563
30-39    30093
20-29    10734
60-69      794
11-19       32
0-10         0
70-79        0
80-89        0
Name: count, dtype: int64


In [146]:
# Load the data BSLG-003
file_path = r"C:\Users\Sarah\Desktop\trend_analysis\data\BSLG-003-JAN-MAR.csv"
df = pd.read_csv(file_path)
print(df.shape)
# Define the ranges
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
labels = ['0-10', '11-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89']

# Create a new column for the ranges
df['Value Range'] = pd.cut(df['value'], bins=bins, labels=labels, right=False)

# Count occurrences in each range
range_counts = df['Value Range'].value_counts()

print(range_counts)


(122921, 2)
Value Range
30-39    65756
20-29    46186
11-19     6754
40-49     4009
0-10       118
50-59       98
60-69        0
70-79        0
80-89        0
Name: count, dtype: int64


In [203]:
import pandas as pd

# Load data BSLG-002
file_path_002 = r"C:\Users\Sarah\Desktop\trend_analysis\data\BSLG-002-JAN-MARCH.csv"
df_002 = pd.read_csv(file_path_002)

# Convert 'time' column to datetime format for proper filtering
df_002['time'] = pd.to_datetime(df_002['time'])

# Extract values within the specified ranges and filter 'time' from 2025-01 to 2025-02
filtered_002 = df_002[
    (((df_002["value"] >= 40) & (df_002["value"] <= 69)) |
    ((df_002["value"] >= 0) & (df_002["value"] <= 29))) 
]

# Count the number of rows in the filtered data
count_filtered = filtered_002.shape[0]
print(f"Number of values retrieved: {count_filtered}")

# Save the filtered data to a CSV file
filtered_file_path = r"C:\Users\Sarah\Desktop\trend_analysis\data\filtered_BSLG-002.csv"
filtered_002.to_csv(filtered_file_path, index=False)
print(f"Filtered data saved to: {filtered_file_path}")


Number of values retrieved: 76261
Filtered data saved to: C:\Users\Sarah\Desktop\trend_analysis\data\filtered_BSLG-002.csv


In [203]:
import pandas as pd

# Load data BSLG-002
file_path_003 = r"C:\Users\Sarah\Desktop\trend_analysis\data\BSLG-003-JAN-MAR.csv"
df_003 = pd.read_csv(file_path_003)

# Ensure 'value' column is in the DataFrame and filter data BSLG-002 by numeric ranges
if 'value' in df_003.columns:
    # Convert 'value' to numeric if needed
    df_003['value'] = pd.to_numeric(df_003['value'], errors='coerce')

    # Filter data within the ranges 1-19, 40-49, and 50-59
    filtered_003 = df_003[
        ((df_003["value"] >= 1) & (df_003["value"] <= 19)) |
        ((df_003["value"] >= 20) & (df_003["value"] <= 29)) 
    ]
    print("Filtered BSLG-003:")
    print(filtered_003)
else:
    print("Column 'value' not found in BSLG-003")
    filtered_003 = pd.DataFrame()

# Load data BSLG-002
file_path_002 = r"C:\Users\Sarah\Desktop\trend_analysis\data\BSLG-002-JAN-MAR.csv"
df_002 = pd.read_csv(file_path_002)

# Check the structure of BSLG-003
if 'value' in df_002.columns:
    print("BSLG-002 loaded successfully:")
    print(df_002)
else:
    print("Column 'value' not found in BSLG-003")

# Combine filtered data from BSLG-002 with all data from BSLG-003
if not filtered_003.empty:
    merged_data = pd.concat([filtered_003, df_002], ignore_index=True)
    print("Merged Data:")
    print(merged_data)
else:
    print("No data from BSLG-002 to merge. Using only data from BSLG-003.")
    merged_data = df_003

# Save to merge.csv
merge_file_path = r"C:\Users\Sarah\Desktop\trend_analysis\data\merge.csv"
merged_data.to_csv(merge_file_path, index=False)

print(f"Data merged and saved as {merge_file_path}")


Filtered BSLG-003:
        value                 time
37      28.35  2025-01-01 00:33:19
38      27.75  2025-01-01 00:34:16
39      27.28  2025-01-01 00:35:21
40      26.24  2025-01-01 00:36:21
41      25.74  2025-01-01 00:36:49
...       ...                  ...
122729  28.72  2025-03-19 21:23:05
122733  28.74  2025-03-19 21:26:40
122734  28.86  2025-03-19 21:27:40
122735  28.90  2025-03-19 21:28:12
122736  28.91  2025-03-19 21:29:14

[43594 rows x 2 columns]
BSLG-002 loaded successfully:
        value                 time
0       43.09  2025-01-01 00:00:21
1       43.63  2025-01-01 00:01:21
2       44.20  2025-01-01 00:02:21
3       44.43  2025-01-01 00:03:21
4       42.67  2025-01-01 00:04:21
...       ...                  ...
111582  32.57  2025-03-19 23:55:26
111583  32.79  2025-03-19 23:56:26
111584  33.15  2025-03-19 23:57:26
111585  33.43  2025-03-19 23:58:26
111586  33.63  2025-03-19 23:59:27

[111587 rows x 2 columns]
Merged Data:
        value                 time
0       28

In [204]:
# Load the data BSLG-003
file_path = r"C:\Users\Sarah\Desktop\trend_analysis\data\merge.csv"
df = pd.read_csv(file_path)
print(df.shape)
# Define the ranges
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
labels = ['0-10', '11-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89']

# Create a new column for the ranges
df['Value Range'] = pd.cut(df['value'], bins=bins, labels=labels, right=False)

# Count occurrences in each range
range_counts = df['Value Range'].value_counts()

print(range_counts)


(155181, 2)
Value Range
20-29    49390
50-59    39371
40-49    30563
30-39    30093
11-19     4852
60-69      794
0-10       118
70-79        0
80-89        0
Name: count, dtype: int64


#  data

In [217]:
#importing required functiond from sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import pandas as pd

In [127]:

#BSLG-002-JAN-MAR
#BSLG-003-JAN-MAR
#merge

,value,time,Value Range
0,43.09,2025-01-01 00:00:21,40-49
1,43.63,2025-01-01 00:01:21,40-49
2,44.2,2025-01-01 00:02:21,40-49
3,44.43,2025-01-01 00:03:21,40-49
4,42.67,2025-01-01 00:04:21,40-49


In [245]:
file_path = r"C:\Users\Sarah\Desktop\trend_analysis\data\merge.csv"
df = pd.read_csv(file_path)
df.head()

,value,time
0,43.09,2025-01-01 00:00:21
1,43.63,2025-01-01 00:01:21
2,44.20,2025-01-01 00:02:21
3,44.43,2025-01-01 00:03:21
4,42.67,2025-01-01 00:04:21


In [246]:

# Define the ranges
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
labels = ['0-10', '11-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89']

df['Value Range'] = pd.cut(df['value'], bins=bins, labels=labels, right=False)
range_counts = df['Value Range'].value_counts()

print(range_counts)

"""
Value Range
50-59    39371
40-49    30563
30-39    30093
20-29    10734
60-69      794
11-19       32
0-10         0
70-79        0
80-89        0
Name: count, dtype: int64
"""


Value Range
30-39    95849
20-29    56920
50-59    39469
40-49    34572
11-19     6786
60-69      794
0-10       118
70-79        0
80-89        0
Name: count, dtype: int64


'\nValue Range\n50-59    39371\n40-49    30563\n30-39    30093\n20-29    10734\n60-69      794\n11-19       32\n0-10         0\n70-79        0\n80-89        0\nName: count, dtype: int64\n'

In [261]:
import pandas as pd

# Assuming df is your DataFrame
# Define the range filter
condition = (df['value'] >= 30) & (df['value'] < 40)

# Limit the number of rows to drop (e.g., up to 100 rows)
rows_to_drop = df[condition].index[:56400]  # Change 100 to your desired number of rows

# Drop the rows
df = df.drop(rows_to_drop)

print("Rows dropped successfully!")
print(df.head())  # Display the first few rows of the updated DataFrame
# Save the updated DataFrame to a CSV file
df.to_csv(r"C:\Users\Sarah\Desktop\trend_analysis\data\merge2.csv", index=False)

Rows dropped successfully!
   value  year  month  day  hour  minute  second  millisecond
0  43.09  2025      1    1     0       0      21            0
2  43.63  2025      1    1     0       1      21            0
3  40.18  2025      1    1     0       1      42            0
4  44.20  2025      1    1     0       2      21            0
5  40.87  2025      1    1     0       2      41            0


In [266]:
file_path = r"C:\Users\Sarah\Desktop\trend_analysis\data\merge2.csv"
df = pd.read_csv(file_path)
df.head()
# Define the ranges
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
labels = ['0-10', '11-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89']

df['Value Range'] = pd.cut(df['value'], bins=bins, labels=labels, right=False)
range_counts = df['Value Range'].value_counts()

print(range_counts)


Value Range
20-29    56920
50-59    39469
30-39    39449
40-49    34572
11-19     6786
60-69      794
0-10       118
70-79        0
80-89        0
Name: count, dtype: int64


In [267]:

print(df.shape)


(178108, 9)


In [ ]:
# column to datetime
df['time'] = pd.to_datetime(df['time'])

# extract time features
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day
df['hour'] = df['time'].dt.hour
df['minute'] = df['time'].dt.minute
df['second'] = df['time'].dt.second
df['millisecond'] = df['time'].dt.microsecond // 1000  # microseconds to milliseconds

# drop time column
df.drop(columns=['time'], inplace=True)
# drop time column
df.drop(columns=['Value Range'], inplace=True)

# Features (X), target variable (y)
X = df.drop(columns=['value'])
y = df['value']

df.head()

In [269]:
# Ensure data is sorted before training
df = df.sort_values(by=['year', 'month', 'day', 'hour', 'minute', 'second', 'millisecond']).reset_index(drop=True)
df.head()


,value,year,month,day,hour,minute,second,millisecond,Value Range
0,43.09,2025,1,1,0,0,21,0,40-49
1,43.63,2025,1,1,0,1,21,0,40-49
2,40.18,2025,1,1,0,1,42,0,40-49
3,44.20,2025,1,1,0,2,21,0,40-49
4,40.87,2025,1,1,0,2,41,0,40-49


In [270]:
# missing values
missing_values = df.isnull().sum()
print("Missing Values per Column:")
print(missing_values)

Missing Values per Column:
value          0
year           0
month          0
day            0
hour           0
minute         0
second         0
millisecond    0
Value Range    0
dtype: int64


In [271]:
# 85% train, 15% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [272]:
# standardize features (x)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [253]:
import joblib

# Save the scaler
joblib.dump(scaler, r"C:\Users\Sarah\Desktop\trend_analysis\model\scaler.pkl")
print("Scaler saved successfully!")


Scaler saved successfully!


In [273]:
# Wrap the scaled arrays with the original feature names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [ ]:
# Initializing base models
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
lr_model = LinearRegression()

# Creating the stacked model
stacked_model = StackingRegressor(estimators=[('rf', rf_model), ('lr', lr_model)])

# train
stacked_model.fit(X_train_scaled, y_train)


In [ ]:
accuracy = stacked_model.score(X_test_scaled, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9029053565770556


In [275]:
# train
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=50,
                              random_state=42)
model.fit(X_train_scaled, y_train)
print(model)


RandomForestRegressor(n_estimators=50, random_state=42)


In [ ]:
from sklearn.model_selection import cross_val_score
# R-squared score
train_score = model.score(X_train_scaled, y_train)
test_score = model.score(X_test_scaled, y_test)

# Cross-validation
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5)

print(f"Train R-squared: {train_score:.4f}")
print(f"Test R-squared: {test_score:.4f}")
print(f"Cross-validation score: {cv_scores.mean():.4f}")


In [259]:
# Predict on test set
y_pred = model.predict(X_test_scaled)

# Show some predictions
df_predictions = pd.DataFrame({'Actual': y_test.values, 'Predicted': y_pred})
print(df_predictions.tail(10))


       Actual  Predicted
46892   31.89    32.2666
46893   30.69    30.7196
46894   43.49    43.4910
46895   51.95    50.1848
46896   23.69    23.7970
46897   55.73    55.8020
46898   31.46    32.2174
46899   31.42    31.4250
46900   32.81    33.8914
46901   51.51    51.6372


In [260]:
import joblib

# Save the trained model
joblib.dump(model, r"C:\Users\Sarah\Desktop\trend_analysis\model\random_forest_model2.pkl")

print("Model saved successfully!")


Model saved successfully!


In [39]:
# Load the model
loaded_model = joblib.load(r"C:\Users\Sarah\Desktop\trend_analysis\model\random_forest_model.pkl")

print("Model loaded successfully!")

# Now you can make predictions using the loaded model
y_pred = loaded_model.predict(X_test)


Model loaded successfully!


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta

# Convert X_test back to DataFrame with column names
X_test_df = pd.DataFrame(X_test, columns=['year', 'month', 'day', 'hour', 'minute', 'second', 'millisecond'])
X_test_df['Actual'] = y_test.values
X_test_df['Predicted'] = y_pred

# Group by day (calculate average actual and predicted value per day)
daily_avg = X_test_df.groupby(['year', 'month', 'day'])[['Actual', 'Predicted']].mean().reset_index()

# Create date column for plotting
daily_avg['date'] = pd.to_datetime(daily_avg[['year', 'month', 'day']])

### 📌 Generate Future Dates for Prediction
future_days = 7  # Predict next 7 days
last_date = daily_avg['date'].max()
future_dates = [last_date + timedelta(days=i) for i in range(1, future_days + 1)]

# Create future DataFrame
future_df = pd.DataFrame({
    'year': [d.year for d in future_dates],
    'month': [d.month for d in future_dates],
    'day': [d.day for d in future_dates],
    'hour': [0] * future_days,  # Assuming midnight for simplicity
    'minute': [0] * future_days,
    'second': [0] * future_days,
    'millisecond': [0] * future_days
})

# Standardize future data
future_scaled = scaler.transform(future_df)

# Predict future water pressure
future_df['Predicted'] = model.predict(future_scaled)
future_df['date'] = future_dates  # Assign future date column

# Append future predictions to daily_avg
extended_df = pd.concat([daily_avg, future_df], ignore_index=True)

### 📌 Plot Actual vs. Predicted (with Future Predictions)
plt.figure(figsize=(12, 6))
sns.lineplot(data=extended_df, x='date', y='Actual', label='Actual', marker='o', linewidth=2, color='blue')
sns.lineplot(data=extended_df, x='date', y='Predicted', label='Predicted (Including Future)', marker='s', linewidth=2, linestyle='dashed', color='red')

# Format the plot
plt.xlabel("Date")
plt.ylabel("Water Pressure")
plt.title("Actual vs. Predicted Water Pressure Trends (with Future Predictions)")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

# Show the plot
plt.show()
